In [2]:
import pandas as pd
import numpy as np
from pybaseball import statcast
from pybaseball import pitching_stats
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [ ]:
def axis_differential(df):
    df['calc_spin'] = np.arctan2(df['pfx_z'],df['pfx_x']) * (180 / np.pi) #FROM ALAN NATHAN MOVEMENT CACULATOR V2
    df['axis_dif'] = np.abs(df['calc_spin'] - df['spin_axis'])
    return df

In [57]:
#The 9P constant acceleration parameters of the trajectory A-I
x0 = 0
y0 = 54
z0 = 5
vx0 = -6.4193
vy0 = -136.7278
vz0 = -5.4231
ax = 12.6634
ay = 31.4401
az = -14.3357

#The 3D spin components J-L
spinx = -1500
spiny = 300
spinz = 1500

spin = np.sqrt((spinx**2)+(spiny**2)+(spinz**2))

yR = 55

#Move the release location to a standard place O-R
tR = (-vy0-np.sqrt(vy0**2-2*ay*(y0-yR)))/ay
vxR = vx0+ax*tR
vyR = vy0+ay*tR
vzR = vz0+az*tR

#Work for next section S-W
t = (-vyR-np.sqrt(vyR**2-2*ay*(yR-17/12)))/ay
vxbar = (2*vxR+ax*t)/2
vybar = (2*vyR+ay*t)/2
vzbar = (2*vzR+az*t)/2
vbar = np.sqrt((vxbar**2)+(vybar**2)+(vzbar**2))
vbar

#Compute the direction of the average velocity X-Z
vx_hat = vxbar/vbar
vy_hat = vybar/vbar
vz_hat = vzbar/vbar

#Work for next section AA-AE
aD = -(vx_hat*ax+vy_hat*ay+vz_hat*(az+32.174))
aTx = ax+aD*vx_hat
aTy = ay+aD*vy_hat
aTz = az+aD*vz_hat+32.174
aT = np.sqrt((aTx**2)+(aTy**2)+(aTz**2))

#Transverse acceleration AF-AH
aTx_hat = aTx/aT
aTy_hat = aTy/aT
aTz_hat = aTz/aT

#Movement angle (Total Movement direction) AI
phiT = np.arctan2(aTz_hat,aTx_hat) * (180/np.pi) #Flipped order from excel version just how its handled by numpy

#Total movement AJ-AL
MTx = 0.5*aTx*t**2*12 #Total X movement
MTz = 0.5*aTz*t**2*12 #Total Z movement
MT = np.sqrt((MTx**2)+(MTz**2))

#Work for next section AM-AP
wx_hat = spinx/spin
wy_hat = spiny/spin
wz_hat = spinz/spin
spin_eff = np.sqrt((wy_hat*vz_hat-wz_hat*vy_hat)**2+(wz_hat*vx_hat-wx_hat*vz_hat)**2+(wx_hat*vy_hat-wy_hat*vx_hat)**2)

#Spin efficiency and the direction of Magnus acceleration AQ-AT
aLx_hat = (wy_hat*vz_hat-wz_hat*vy_hat)/spin_eff
aLy_hat = (wz_hat*vx_hat-wx_hat*vz_hat)/spin_eff
aLz_hat = (wx_hat*vy_hat-wy_hat*vx_hat)/spin_eff
phiL = np.arctan2(aLz_hat,aLx_hat) * (180/np.pi) #Magnus movement direction

#Axis shift AU-AV
axis_shift = phiT-phiL #Axis shift
T_dot_L = (aTx_hat*aLx_hat+aTy_hat*aLy_hat+aTz_hat*aLz_hat)

#CL using lookup table AW-AX
ML = MT*T_dot_L
MS = np.sqrt(MT**2-ML**2)

#Work for next section AY-AZ
MLx = ML*aLx_hat
MLz = ML*aLz_hat

#Compute Magnus BA-BC
MSx = MTx - MLx
MSz = MTz - MLz

#Non-Magnus Direction BG
phiS = np.arctan2(MSz,MSx) * (180/np.pi)

134.65454155167

In [ ]:
"""
Template does all the necessary calculations, with additional input from K and f(S).
There are two templates:
M-NM separates into Magnus and non-Magnus.  It requires knowledge of Magnus lift coefficient.
L-S separates into Lift and Side components.  It does not require knoweldge of Magnus lift coefficien.

Input to both templates, highlighted in orange:
columns A-I:  The 9P constant acceleration parameters of the trajectory
columns J-L:  The 3D spin components
---------------------------------------------------------------------------------
Calculation in M-NM, highlighted in blue:
First move the release location to a standard place, which is !K$B13, currently set at 55 ft (columns O-R)
Next compute the direction of the average velocity (X-Z) and the transverse acceleration (AF-AH)
Next compute the total movement (AJ-AL) and movement angle (AI)
Next compute the spin efficiency and the direction of the Magnus acceleration (AQ-AT) and the Magnus direction and axis shift (AU-AV).
Next use lookup table in f(S) to find CL (AW-AX)
Next compute the Magnus (BA-BC) and non-Magnus (BD-BF) movement and the non-Magnus direction (BG)

To get the lift coefficient requires knowing the mass and size of the ball and the atmospherics, which is done in K, with result !KD3. 

The lift coefficient uses the lookup table in worksheet f(S), based on the value of S*spin_eff in  AX, returning the CL value in AY.
The table is based on my current best shot at determining the relationship between CL and S.  The table will be updated as more information is gathered.
-----------------------------------------------------------------------------------------------
Similar remarks apply to L-S.  The primary results of interest are:
Total movement:  AJ,AK; Total movement direction: AI
Lift movement:  AY,AZ; Lift movement direction:  AT
Side movement:  BA,BB; Side movement direction:  BC
Spin efficiency:  AP
"""
